### Fase 2: Extracción de Detalles de Películas con Selenium
Utiliza Selenium para obtener información adicional de las películas listadas previamente.

**Información requerida:**
- Calificación de IMDB.
- Dirección (director o directores).
- Guionistas.
- Argumento.
- Duración (en minutos).

**Nota:** Los datos obtenidos deberán almacenarse en una lista de tuplas.

---

## Importamos librerías y configuraciones

In [1]:
%load_ext autoreload
%autoreload 2

from config import *

---
---
## cargamos datos

In [2]:
with open(f'{DATOS_RUTA_BASE}/movie_shorts_list_flated.pkl', 'rb') as f:
     movies_shorts_list = pickle.load(f)

In [3]:
movies_shorts_list[:2]

[('movie', "S. Lubin's Passion Play", '1903-1', 'tt11166772'),
 ('movie', 'The Story of the Kelly Gang', '1906-12', 'tt0000574')]

---
---
## Obtenemos Información Adicional de la película

#### para esto, voy a utilizar `BeautifulSup` ya que es más rápido que Selenium y como ya tenemos las id de las películas, podemos buscar por la URL y obtener toda la información que necesitamos de una sola página sin interactuar con nada de la misma.

In [6]:
from bs4 import BeautifulSoup

#### comprobamos duplicados y eliminamos los que tenemos

In [5]:
len(movies_shorts_list)

28000

In [7]:
df = pd.DataFrame(movies_shorts_list, columns=['tipo', 'nombre', 'fecha', 'id'])

In [10]:
df.head()

,tipo,nombre,fecha,id
0,movie,S. Lubin's Passion Play,1903-1,tt11166772
1,movie,The Story of the Kelly Gang,1906-12,tt0000574
2,movie,Robbery Under Arms,1907-11,tt0000615
3,movie,L'enfant prodigue,1907-6,tt0000591
4,movie,Violante,0-0,tt0176294


In [9]:
df.duplicated().sum()

27251

In [11]:
df.drop_duplicates(inplace=True)

In [13]:
df.shape

(749, 4)

In [20]:
movie_shorts_noduplicated = list(df.itertuples(index=False, name=None))

#### guardamos nuestra tupla sin duplicados

In [21]:
with open(f'{DATOS_RUTA_BASE}/movie_shorts_noduplicated.pkl', 'wb') as f:
    pickle.dump(movie_shorts_noduplicated, f)

---
---
## comenzamos con las llamadas http para obtener la información adicional

In [79]:
# url = f'https://www.imdb.com/es-es/title/tt0356838/'

# soup = BeautifulSoup(requests.get(url, headers={"User-Agent": "Chrome"}).content, 'html.parser')

In [ ]:
# try:
#     rate = soup.find('div', {'class': 'sc-d541859f-2 kxphVf'}).get_text()
# except:
#     rate = 'unknown'

# rate

'6,7/10'

In [ ]:
# try:
#     li_directors = soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'}).find_all('a')
#     directors = [item.get_text() for item in li_directors]
#     directors
# except:
#     directors = 'unknown'

# directors

['Alfred Rolfe']

In [ ]:
# try:
#     li_writers = soup.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[1].find_all('a')
#     writers = [item.get_text() for item in li_writers]
#     writers
# except:
#     writers = 'unknown'

# writers

['Arthur Wright']

In [112]:
# try:
#     argument = soup.find('span', {'class' : 'sc-3ac15c8d-2 fXTzFP'}).get_text()
# except:
#     argument = 'unknown'

# argument

'The story of the Eureka Stockade, where 12,000 gold miners rose up against the government.'

In [118]:
# try:
#     duration = soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt'}).find_all('li')[1].get_text()
# except:
#     duration = '0'

# if duration.find('h') != -1:
#     duration = int(duration.replace('h','')) * 60

# duration

60

## funcion para el 'scrapeo'

In [124]:
def get_movie_short_details(register):
    url = f'https://www.imdb.com/es-es/title/{register[3]}/'

    soup = BeautifulSoup(requests.get(url, headers={"User-Agent": "Chrome"}).content, 'html.parser')

    try:
        rate = soup.find('div', {'class': 'sc-d541859f-2 kxphVf'}).get_text()
    except:
        rate = 'unknown'


    try:
        li_directors = soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'}).find_all('a')
        directors = [item.get_text() for item in li_directors]
        directors
    except:
        directors = 'unknown'


    try:
        li_writers = soup.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[1].find_all('a')
        writers = [item.get_text() for item in li_writers]
        writers
    except:
        writers = 'unknown'


    try:
        argument = soup.find('span', {'class' : 'sc-3ac15c8d-2 fXTzFP'}).get_text()
    except:
        argument = 'unknown'


    try:
        duration = soup.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-ec65ba05-2 joVhBE baseAlt'}).find_all('li')[1].get_text()
    except:
        duration = '0'

    if duration.find('h') != -1:
        duration = int(duration.split('h')[0]) * 60


    return register + (rate, directors, writers, argument, duration, )


In [125]:
movies_shorts_detailed = list(map(get_movie_short_details, movie_shorts_noduplicated))

In [127]:
movies_shorts_detailed[:3]

[('movie',
  "S. Lubin's Passion Play",
  '1903-1',
  'tt11166772',
  'unknown',
  'unknown',
  'unknown',
  '',
  60),
 ('movie',
  'The Story of the Kelly Gang',
  '1906-12',
  'tt0000574',
  '6,0/10',
  ['Charles Tait'],
  ['Charles Tait'],
  'Originalmente de 70 minutos de duración, sólo 17 minutos del primer largometraje narrativo del mundo sobrevivieron en fotogramas y otros fragmentos y cuentan la historia de Ned Kelly, un infame forajido australiano del siglo XIX.',
  'Not Rated'),
 ('movie',
  'Robbery Under Arms',
  '1907-11',
  'tt0000615',
  '4,3/10',
  ['Charles MacMahon'],
  ['Rolf Boldrewood', 'Charles MacMahon'],
  '',
  '0')]

---
---
## guardamos los datos

In [126]:
with open(f'{DATOS_RUTA_BASE}/movies_shorts_detailed.pkl', 'wb') as f:
    pickle.dump(movies_shorts_detailed, f)